This project will involve the creation of a spellchecking system and an evaluation of its performance.
Please start by downloading the corpus `holbrook.txt` from Data.

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
You do not need to separate these words, but instead you may treat them like a single token.

## Task 1

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

## Note: 
* For the course of this assignment, all the calculations are done by converting words into lower case.

In [379]:
from nltk.tokenize import word_tokenize
lines = open("data/holbrook.txt").readlines()
data = []

for line in lines:
    words=word_tokenize(line)
    #preparing dictionary with original and corrected sentences. 
    dict={}
    dict['original']=[w[0:w.index('|')] if '|' in w else w for w in words ]
    dict['corrected']=[w[w.index('|')+1:len(w)] if '|' in w else w for w in words ]
    dict['indexes']=[words.index(w) for w in words if '|' in w]
    data.append(dict)

assert(data[2] == {
    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
    'indexes': [9]
})

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [380]:
#splitting data into train and test data sets.
test = data[:100]
train = data[100:]

## **Task 2**
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [400]:
from collections import Counter
import itertools

#Global variable for storing all words in corrected train data sets.
all_words=[]

#creating all words from corrected train data set
def create_all_words(data_set):
    words=list(itertools.chain.from_iterable([d['corrected'] for d in data_set]))
    [all_words.append(w.lower()) for w in words]

#find frequency of a given word
def unigram(word):
    return Counter(all_words)[word.lower()]      

#find unigram probability of a given word
def prob(word):
    words_with_frequency=Counter(all_words)
    return words_with_frequency[word]/sum([words_with_frequency[freq] for freq in words_with_frequency])

create_all_words(train)
assert(unigram("me")==87)

## **Task 3**: 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [404]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("there", "their"))

2


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [455]:
def get_candidates(token):
    unique_tokens=set(all_words)
    dist={}
    for word in unique_tokens:
        measure=edit_distance(token.lower(),word)
        dist[word]=measure
    
    #returning words with a min edit distance value
    return [k for k,v in dist.items() if v==min(dist.values())]

#Test your code as follows
assert get_candidates("minde") == ['mine', 'mind']

## Task 4:

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [458]:
from copy import deepcopy

def correct(sentence):
    
    #making copy of a original sentence
    new_sentence=deepcopy(sentence)
    
    i=0
    while i<len(new_sentence):
        #dict to store unigram probabilities of matching words found by edit distance
        unigram_probabilities={}
        
        if new_sentence[i].lower() not in set([w.lower() for w in all_words]):
            
            #get similar candidates for each of the word
            similar_words =get_candidates(new_sentence[i])
            
            #iterate and find unigram probability of each similar word
            for similar_word in similar_words:
                unigram_probabilities[similar_word]=prob(similar_word)
                
            #get correct word by finding max of all unigram probabilities
            correct_word=max(unigram_probabilities, key=unigram_probabilities.get)
            
            #update sentence
            new_sentence[i]=correct_word
            
        i=i+1
        
    return new_sentence
assert(correct(["this","whitr","cat"]) == ['this','white','cat'])   

## **Task 5**: 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [457]:
def accuracy(test):
    correct_word_count=0
    total_word_count=0
    
    #looping over test data set to find the corrected sentence
    for sent in test:
        
        #calculate total words in corrected data data set
        total_word_count=total_word_count+len(sent['original'])
        
        #get the corrected sentence
        result=correct(sent['original'])
        
        #get correct word count
        correct_word_count=correct_word_count+get_correct_word_count(sent['corrected'],result)
    #finding accuracy
    return (correct_word_count/total_word_count)*100

#function to find no of correct words from test data set
def get_correct_word_count(test_data_sent,corrected_sent):
    i=0
    correct_word_count=0
    while i<len(test_data_sent):
        if test_data_sent[i].lower()==corrected_sent[i].lower():
            correct_word_count=correct_word_count+1
        i=i+1
    return correct_word_count

print("Accuracy: "+str(accuracy(test)))


Accuracy: 84.0669014084507


## **Task 6:**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code


### Solution:

* nb_prob() finds probability of a word using Noisy Channel Model.
        p(C|W)=>p(W|C)*p(C)
* Applying Laplace smoothing to the assign probability.
* get_candidates_updated() normalises edit_distance by sum of the length of both the words. Normalizing the edit distance is one way to put the differences between strings on a single scale.

In [472]:
def get_candidates_updated(token):
    unique_tokens=set(all_words)
    dist={}
    for word in unique_tokens:
        measure=edit_distance(token.lower(),word)
        #normalising before assinging it to dict
        dist[word]=measure/(len(token.lower())+len(word))
    
    #returning words with a min edit distance value
    return [k for k,v in dist.items() if v==min(dist.values())]

#Applying probability according to the Noisy Channel Model
#Probability of a similar word(c) given existing word(w)
def nb_prob(c,w):
    n=0
    d=0
    unique_tokens=set(all_words)
    for data in train:
        i=0
        while i<len(data['corrected']):
            if data['corrected'][i].lower()==c.lower():
               d=d+1
               if data['original'][i].lower()==w.lower():
                   n=n+1
            i=i+1
    #applying smoothing with alpha taken as 0.5
    return (n+0.5/(d+0.5*(len(unique_tokens))))*prob(c)

def correct_updated(sentence):
    
    #making copy of a original sentence
    new_sentence=deepcopy(sentence)
    
    i=0
    while i<len(new_sentence):
        #dict to store probabilities of matching words found by edit distance
        nb_probabilities={}
        
        if new_sentence[i].lower() not in set([w.lower() for w in all_words]):
            
            #get similar candidates for each of the word
            similar_words =get_candidates_updated(new_sentence[i])
            
            if len(similar_words)>0:
                #iterate and find probability of a given word acc to noisy channel model
                for similar_word in similar_words:
                    nb_probabilities[similar_word]=nb_prob(similar_word,new_sentence[i])

                #get correct word by finding max of all probabilities
                correct_word=max(nb_probabilities, key=nb_probabilities.get)

                #update sentence
                new_sentence[i]=correct_word
            
        i=i+1
        
    return new_sentence

## **Task 7:**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [473]:
def accuracy(test):
    correct_word_count=0
    total_word_count=0
    
    #looping over test data set to find the corrected sentence
    for sent in test:
        
        #calculate total words in corrected data data set
        total_word_count=total_word_count+len(sent['original'])
        
        #get the corrected sentence
        result=correct_updated(sent['original'])
        
        #get correct word count
        correct_word_count=correct_word_count+get_correct_word_count(sent['corrected'],result)
    #finding accuracy
    return (correct_word_count/total_word_count)*100

print("Accuracy: "+str(accuracy(test)))

Accuracy: 85.47535211267606


### Conlcusion:

As clearly seen from the observation, accuracy of a modified algorithm is greater than the existing one.